# Récupération des données avec l'API Fred

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import json
#import pour l'API
import requests
from statistics import mean
import time

Création d'un dictionnaire contenant tous les états et des sous-dictionnaires pour chaque comtés

In [ ]:
#définition de l'url et des paramètres de la première requête
api_key = "180de2e6a1d1e953d270ebf38341cd44"
param = {"api_key" : api_key, "file_type" : "json", "category_id" : "27281"}
url = "https://api.stlouisfed.org/fred/category/children?"
response = requests.get(url, params=param)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    # Process the data as needed
else:
    print(f"Error: {response.status_code}, {response.text}")

# création d'un dictionnaire avec les états pour clés et leur catégorie counties pour valeur
liste = data["categories"] #on rentre dans le premier dictionnaire
dico_id_states = dict()

for dic in liste : #on parcourt la liste en stockant dans un dictionnaire le nom de l'état et le numéro de la catégorie associée
    dico_id_states[dic["name"]] = dic["id"]

state_relous = ["Alaska", "Louisiana","Virgin Islands"] #on retire les états qui n'ont pas de comties
for state in state_relous :
    del dico_id_states[state]

for state in dico_id_states.keys() : #on parcourt les états
    param["category_id"] = dico_id_states[state] #on ajuste les paramètres de la request pour demander la bonne catégorie
    response = requests.get(url, params = param)
    data = response.json()
    dic = data["categories"][0] #on rentre dans le dictionnaire (toujours le premier, c'est celui qui contient les comtés)

    category_id = dic["id"] #on suavegarde la catégorie contenant les comtés de l'état
    dico_id_states[state] = category_id

dic_states_counties = dict() #on initialise un nouveau dictionnaire, qui sera de la forme : {etat1 : {comté1 : {}, comté2 :}, état2 : ....}

for state in dico_id_states.keys() : #on parcourt les états
    param["category_id"] = dico_id_states[state] #on ajuste les paramètres de la request pour demander la bonne catégorie
    response = requests.get(url, params = param)
    data = response.json()
    liste = data["categories"] #on obtient la liste des comtés de l'état

    dic_states_counties[state] = dict() #on prépare le dictionnaire qui contiendra aura les comtés de l'état pour clés
    

    for dic in liste : #on parcourt les dictionnaires (1 par county)
        county_name = dic["name"] #on récupère le nom du county
        dic_states_counties[state][county_name] = {"id" : dic["id"]} #on crée une entrée dans le dic pour le county et on sauvegarde sa catégorie

counties_df = pd.DataFrame() #on  créé le dataframe qui servira à tout sauvegarder et à convertir en csv

for state in dic_states_counties.keys(): #on va parcourir état par état (car c'est les premières clés)
    dic_state = dic_states_counties[state] #on se place dans le dictionnaire de l'état

    for county in dic_states_counties[state].keys(): #on parcourt les counties
        dic_county = dic_state[county] #on se place dans le dictionnaire du county

        url = "https://api.stlouisfed.org/fred/category/series?"
        param = {"api_key" : api_key, "file_type" : "json", "category_id" : dic_county["id"]} #on va accéder à la page contenant les séries du county

        response = requests.get(url, params= param)
        data = response.json()     
        
        #maintenant on cherche la série dont le titre contient "unemployment rate" et dont la fréquence est annuelle
        liste_series = data["seriess"]
        
        for serie in liste_series: #on parcourt les séries,en cherchant celle qui parle du taux de chomage annuel
            if ("Unemployment Rate" in serie["title"]) and ("Monthly" in serie["frequency"]):
                dic_county["unemployment_rate_id"] = serie["id"]  #on récupère l'id de la série et on le stock
        time.sleep(0.5)

    for county in dic_states_counties[state].keys(): #on parcourt les counties
        dic_county = dic_state[county] #on se place dans le dictionnaire du county

        #récupération de la série
        if "unemployment_rate_id" in dic_county.keys() :

            url = "https://api.stlouisfed.org/fred/series/observations?" #on va chercher les séries correspondant au chomage mensuel
            param = {"api_key" : api_key, "file_type" : "json", "series_id" : dic_county["unemployment_rate_id"], "observation_start" : "2013-01-01", "observation_end" : "2018-03-01"} 
        
            response = requests.get(url, params= param)
            data = response.json()

            liste_observations = [float(obs["value"]) for obs in data["observations"]] #on récupère toutes les observations pour en faire la moyenne sur la période
            
            unemployment_rate_mean = mean(liste_observations)

            dic_county["state"] = state
            dic_county["unemployment_rate"] = unemployment_rate_mean

        time.sleep(0.5)

    df_temp = pd.DataFrame.from_dict(dic_states_counties[state], orient = "index")
    counties_df = pd.concat([counties_df, df_temp])

counties_df.to_csv(path_or_buf="counties_data2.csv")